In [1]:
import numpy as np
import pandas as pd
import nbimporter
import os

from IPython.display import Image, display
from sklearn.linear_model import LogisticRegression
from nltk.metrics import *
from nltk.tag import *
from nltk.parse.bllip import BllipParser
from nltk import Tree
from nltk.draw.util import CanvasFrame
from nltk.draw import TreeWidget
from nltk.corpus import treebank
from sent_utils import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout

Importing Jupyter notebook from sent_utils.ipynb


In [2]:
init_notebook_mode(connected=True)
MODEL_PATH = find('models/bllip_wsj_no_aux').path
PARSER = BllipParser.from_unified_model_dir(MODEL_PATH)

In [13]:
# Paraphrase detectors
def paraphrase_score(sent_0, sent_1):
    print('.', end='')
    sent_0, sent_1 = remove_punctuation(sent_0.lower()), remove_punctuation(sent_1.lower())
    words_0, words_1 = words_from_sent(sent_0), words_from_sent(sent_1)
    tokens_0, tokens_1 = tokens_from_words(words_0), tokens_from_words(words_1)
    lemmas_0, lemmas_1 = tokens_to_lemmas(tokens_0), tokens_to_lemmas(tokens_1)
    senses_0, senses_1 = tokens_to_senses(tokens_0), tokens_to_senses(tokens_1)
#     pos_0, pos_1 = str(PARSER.parse_one(sent_0.split())), str(PARSER.parse_one(sent_1.split()))
    return [
        jaccard_distance(set(lemmas_0), set(lemmas_1))
#         ,
#         pos_0.count('NP'),
#         pos_0.count('VP'),
#         pos_1.count('NP'),
#         pos_1.count('VP')
    ]

In [ ]:
train_input = text_to_sentences('IHLT-eval-framework/train/msr_paraphrase_train_input.txt')
X_train = [paraphrase_score(data[0], data[1])for data in train_input]
y_train = open('IHLT-eval-framework/train/msr_paraphrase_train_gs.txt', encoding="utf-8-sig").readlines()
y_train = [int(line.strip()) for line in y_train]

test_input = text_to_sentences('IHLT-eval-framework/test/msr_paraphrase_test_input.txt')
X_test = [paraphrase_score(data[0], data[1])for data in test_input]
y_test = open('IHLT-eval-framework/test/msr_paraphrase_test_gs.txt', encoding="utf-8-sig").readlines()
y_test = [int(line.strip()) for line in y_test]

In [ ]:
regression = LogisticRegression()
regression.fit(np.array(X_train), y_train)
prediction = regression.predict(np.array(X_test))
MSRP_eval(prediction, y_test)

In [7]:
false_negative_idx = list(np.where((prediction != y_test) & (prediction == 0))[0])
false_negatives = [input_sentences[i] for i in false_negative_idx]

In [ ]:
for false_negative in false_negatives:
    sent_0, sent_1 = false_negative[0], false_negative[1]
    pos_0, pos_1 = PARSER.parse_one(sent_0.split()), PARSER.parse_one(sent_1.split())
    str(pos_0).count('NP')